<a href="https://colab.research.google.com/github/promaprogga/iResSENet-An-Accurate-Convolutional-Neural-Network-for-Retinal-Blood-Vessel-Segmentation/blob/main/DRIVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.8.2


In [ ]:

from platform import python_version
  
  
print("Current Python Version-", python_version())

Current Python Version- 3.7.14


In [ ]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import HorizontalFlip, VerticalFlip, ElasticTransform, GridDistortion, OpticalDistortion, Rotate, CenterCrop, RandomRotate90, Transpose, RandomSizedCrop, Compose, OneOf,PadIfNeeded, CLAHE, RandomBrightnessContrast, RandomGamma

## Data Augmentation

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def augment_data(images, masks, save_path, augment=True):
    H = 512
    W = 512

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
       
        name = x.split("/")[-1].split(".")[0]
        
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = imageio.mimread(y)[0]
        # print(x.shape, y.shape)

        if augment == True:
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]

        
            aug = ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03)
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = augmented['mask']

            aug = GridDistortion(p=1)
            augmented = aug(image=x, mask=y)
            x4 = augmented['image']
            y4 = augmented['mask']

            aug = OpticalDistortion(distort_limit=2, shift_limit=0.5, p=1)
            augmented = aug(image=x, mask=y)
            x5 = augmented['image']
            y5 = augmented['mask']


            aug = OpticalDistortion(distort_limit=2, shift_limit=0.5, p=1)
            augmented = aug(image=x, mask=y)
            x5 = augmented['image']
            y5 = augmented['mask']

            aug = CenterCrop(p=1, height=H, width=W)
            augmented = aug(image=x, mask=y)
            x6 = augmented['image']
            y6 = augmented['mask']


            aug = RandomRotate90(p=1)
            augmented = aug(image=x, mask=y)
            x7 = augmented['image']
            y7 = augmented['mask']

            aug = Transpose(p=1)
            augmented = aug(image=x, mask=y)
            x8 = augmented['image']
            y8 = augmented['mask']

            aug = RandomSizedCrop(min_max_height=(50, 101), height=H, width=W, p=1)
            augmented = aug(image=x, mask=y)
            x9 = augmented['image']
            y9 = augmented['mask']

            aug = Compose([VerticalFlip(p=0.5),RandomRotate90(p=0.5)])
            augmented = aug(image=x, mask=y)
            x10 = augmented['image']
            y10 = augmented['mask']


            aug = Compose([
                           OneOf([
                                  RandomSizedCrop(min_max_height=(50, 101), height=H, width=W, p=0.5),
                                  PadIfNeeded(min_height=H, min_width=W, p=0.5)],p=1),
                           VerticalFlip(p=0.5),
                           RandomRotate90(p=0.5),
                           OneOf([
                                  ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
                                  GridDistortion(p=0.5),
                                  OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1),], p=0.8)])
            
            augmented = aug(image=x, mask=y)
            x11 = augmented['image']
            y11 = augmented['mask']


            aug = Compose([
                           OneOf([
                                  RandomSizedCrop(min_max_height=(50, 101), height=H, width=W, p=0.5),
                                  PadIfNeeded(min_height=H, min_width=W, p=0.5)],p=1),
                           VerticalFlip(p=0.5),
                           RandomRotate90(p=0.5),
                           OneOf([
                                  ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5),
                                  GridDistortion(p=0.5),
                                  OpticalDistortion(distort_limit=2, shift_limit=0.5, p=1)], p=0.8),
                           CLAHE(p=0.8),
                           RandomBrightnessContrast(p=0.8),    
                           RandomGamma(p=0.8)])
            
            augmented = aug(image=x, mask=y)
            x12 = augmented['image']
            y12 = augmented['mask']



            
            X = [x, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12]
            Y = [y, y1, y2, y3, y4, y5, y6, y7, y8, y9, y10, y11, y12]


        else:
            X = [x]
            Y = [y]

        index = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            tmp_image_name = f"{name}_{index}.jpg"
            tmp_mask_name = f"{name}_{index}.jpg"
            image_path = os.path.join(save_path, "image", tmp_image_name)
            mask_path = os.path.join(save_path, "groundtruth", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            index += 1
            


In [ ]:
def load_data(path):
    #X = Images and Y = masks

    train_x = sorted(glob(os.path.join(path, "training", "images", "*.tif")))
    train_y = sorted(glob(os.path.join(path, "training", "1st_manual", "*.gif")))

    test_x = sorted(glob(os.path.join(path, "test", "images", "*.tif")))
    test_y = sorted(glob(os.path.join(path, "test", "1st_manual", "*.gif")))

    return (train_x, train_y), (test_x, test_y)




""" Seeding """
np.random.seed(42)

""" Load the data """
data_path = "/content/drive/MyDrive/Dataset/DRIVE"
(train_x, train_y), (test_x, test_y) = load_data(data_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")



In [ ]:
create_dir("/content/drive/MyDrive/Dataset/DRIVE/new_data/train/image/")
create_dir("/content/drive/MyDrive/Dataset/DRIVE/new_data/train/groundtruth/")
create_dir("/content/drive/MyDrive/Dataset/DRIVE/new_data/test/image/")
create_dir("/content/drive/MyDrive/Dataset/DRIVE/new_data/test/groundtruth/")


augment_data(train_x, train_y, "/content/drive/MyDrive/Dataset/DRIVE/new_data/train/", augment=True)
augment_data(test_x, test_y, "/content/drive/MyDrive/Dataset/DRIVE/new_data/test/", augment=False)

In [ ]:
def load_data1(path):
    #X = Images and Y = masks

    train_x = sorted(glob(os.path.join(path, "train", "image", "*.jpg")))
    train_y = sorted(glob(os.path.join(path, "train", "groundtruth", "*.jpg")))

    test_x = sorted(glob(os.path.join(path, "test", "image", "*.jpg")))
    test_y = sorted(glob(os.path.join(path, "test", "groundtruth", "*.jpg")))

    return (train_x, train_y), (test_x, test_y)
np.random.seed(42)

data_path = "/content/drive/MyDrive/Dataset/DRIVE/new_data"
(train_x, train_y), (test_x, test_y) = load_data1(data_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

## Model

In [ ]:

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Concatenate, Input
from tensorflow.keras.models import Model
from keras import optimizers, losses
from keras.layers import *
from keras.models import Model
from keras.backend import int_shape
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from albumentations import CenterCrop, RandomRotate90, GridDistortion, HorizontalFlip, VerticalFlip
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, BatchNormalization, Bidirectional, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, ZeroPadding2D,ConvLSTM2D,LSTM,GlobalAveragePooling2D, Reshape, Dense, Multiply, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import UpSampling2D, Input, Concatenate,TimeDistributed, Add
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau,EarlyStopping
from tensorflow.keras.applications import InceptionResNetV2,MobileNetV2,ResNet50,VGG16,VGG19, NASNetLarge,EfficientNetB7,EfficientNetV2M
from tensorflow.keras.metrics import Recall, Precision, Accuracy, MeanIoU
from tensorflow.keras import backend as K



def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("gelu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("gelu")(x)

    return x

    
def se_block(block_input, num_filters, ratio=8):                             # Squeeze and excitation block

	pool1 = GlobalAveragePooling2D()(block_input)
	flat = Reshape((1, 1, num_filters))(pool1)
	dense1 = Dense(num_filters//ratio, activation='gelu')(flat)
	dense2 = Dense(num_filters, activation='sigmoid')(dense1)
	scale = multiply([block_input, dense2])
	
	return scale

def resnet_block(block_input, num_filters):                                  # Single ResNet block


	if int_shape(block_input)[3] != num_filters:
		block_input = Conv2D(num_filters, kernel_size=(1, 1))(block_input)
	
	conv1 = Conv2D(num_filters, kernel_size=(1, 1), padding='same')(block_input)
	norm1 = BatchNormalization()(conv1)
	relu1 = Activation('gelu')(norm1)
 
	conv2 = Conv2D(num_filters, kernel_size=(3,3), padding='same')(relu1)
	norm2 = BatchNormalization()(conv2)
	
	se = se_block(norm2, num_filters=num_filters)
 	sum = Add()([block_input, se])
	relu2 = Activation('gelu')(sum)
	
	return relu2


def encoder_block(inputs, num_filters, length):
    x = resnet_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
  
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    p0 = inputs
    s1, p1 = encoder_block(p0, 32, 4)
    s2, p2 = encoder_block(p1, 64, 3)
    s3, p3 = encoder_block(p2, 128, 2)
    s4, p4 = encoder_block(p3, 256, 1)

    b1 = conv_block(p4, 512)
   

    d1 = decoder_block(b1, s4, 256)
    d2 = decoder_block(d1, s3, 128)
    d3 = decoder_block(d2, s2, 64)
    d4 = decoder_block(d3, s1, 32)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="iResSENet")
    return model


input_shape = (512, 512, 3)
model = build_unet(input_shape)
model.summary()

## Dice Loss

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)



## Training

In [ ]:

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision, AUC
from sklearn import metrics 

H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "groundtruth", "*.jpg")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

    
""" Directory to save files """
create_dir("/content/drive/MyDrive/Dataset/DRIVE/filesdrive")

""" Hyperparameters """
batch_size = 2
lr = 1e-4
num_epochs = 50

path="/content/drive/MyDrive/Dataset/DRIVE/filesdrive"
model_path = os.path.join(path, "model.h5")
csv_path = os.path.join(path, "data.csv")

""" Dataset """
dataset_path = "/content/drive/MyDrive/Dataset/DRIVE/new_data"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")

train_x, train_y = load_data(train_path)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
# print(train_dataset)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_setps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
        train_steps += 1
if len(valid_x) % batch_size != 0:
        valid_setps += 1

""" Model """
input_shape=(H,W,3)
model=build_unet(input_shape)
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
# model.summary()

callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, min_lr=1e-6, patience=5, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
    ]

model.fit(
      train_dataset,
      epochs=num_epochs,
      validation_data=valid_dataset,
      steps_per_epoch=train_steps,
      validation_steps=valid_setps,
      callbacks=callbacks
    )

## Evaluation

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score, roc_auc_score

H = 512
W = 512
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)
    return ori_x, x
    
def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "groundtruth", "*.jpg")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)


""" Save the results in this folder """
create_dir("/content/drive/MyDrive/Dataset/DRIVE/resultsdrive")

""" Load the model """
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
   model = tf.keras.models.load_model("/content/drive/MyDrive/Dataset/DRIVE/filesdrive/model.h5")

""" Load the dataset """
dataset_path = os.path.join("/content/drive/MyDrive/Dataset/DRIVE/new_data", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """

SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        #  Extracting name
        name = x.split("/")[-1].split(".")[0]

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Prediction """
        y_pred = model.predict(np.expand_dims(x, axis=0))[0]
        y_pred = y_pred > 0.5
        # print(y_pred.shape)
        y_pred = y_pred.astype(np.int32)
        y_pred = np.squeeze(y_pred, axis=-1)

        """ Saving the images """
        save_image_path = f"/content/drive/MyDrive/Dataset/DRIVE/resultsdrive/{name}.png"
        save_results(ori_x, ori_y, y_pred, save_image_path)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculate the metrics """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
        auc =  roc_auc_score(y, y_pred,  multi_class='ovr')

        SCORE.append([name, acc_value,f1_value, jac_value, recall_value, precision_value ,auc ])


score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.4f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.4f}")


""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision","Auc"])
df.to_csv("/content/drive/MyDrive/Dataset/DRIVE/filesdrive/score.csv")
df.plot(figsize=(10,8))
